In [253]:
import cvxpy
import random
import numpy as np
import pandas as pd

In [254]:
# サンプルデータの読み込み
sample = pd.read_csv('sample.csv')

In [255]:
sample.head(3)

,id,price_A,price_B,price_C,rating_A,rating_B,rating_C
0,1,380,385,389,4.1,3.3,4.0
1,2,943,947,961,10.1,8.9,9.3
2,3,980,980,987,10.4,8.7,10.3


In [256]:
# クライアトa, b, cの元々持っていた枠をランダムに決める。
length = len(sample)
ids = list(range(length))
# idは1からなので、1を足す。
random_ids = [id+1 for id in ids]

# Break Point 1の略。
bp1 = length//3 
bp2 = length//3 + bp1

random.shuffle(random_ids)
a, b, c = random_ids[:bp1], random_ids[bp1:bp2], random_ids[bp2:]

In [257]:
# それぞれのidを表示
print(a)
print(b)
print(c)

[14, 8, 22, 27, 24, 10, 12, 1, 18, 7]
[13, 30, 26, 29, 15, 19, 5, 16, 20, 2]
[17, 25, 28, 3, 4, 6, 23, 11, 9, 21]


In [258]:
# それぞれのidのデータのみを取り出すための判別用カラム。
sample['judge_a'] = sample['id'].apply(lambda x:x in a)
sample['judge_b'] = sample['id'].apply(lambda x:x in b)
sample['judge_c'] = sample['id'].apply(lambda x:x in c)

In [259]:
# それぞれのidのデータを取得。
df_a = sample[sample['judge_a']].loc[:, ['id','price_A', 'rating_A']]
df_b = sample[sample['judge_b']].loc[:, ['id','price_B', 'rating_B']]
df_c = sample[sample['judge_c']].loc[:, ['id','price_C', 'rating_C']]

In [260]:
# 判別用カラムを削除。
del sample['judge_a']
del sample['judge_b']
del sample['judge_c']

In [261]:
# 元々の値段の総和、元々の視聴率の和を計算する。
# Original price, Original point
o_price_a, o_point_a = sum(df_a['price_A']), sum(df_a['rating_A'])
o_price_b, o_point_b = sum(df_b['price_B']), sum(df_b['rating_B'])
o_price_c, o_point_c = sum(df_c['price_C']), sum(df_c['rating_C'])

In [262]:
# 獲得した枠の価格を貯めていく。
obtained_price_A = 0
obtained_price_B = 0
obtained_price_C = 0

In [263]:
# 獲得した枠の視聴率を貯めていく。
obtained_point_A = 0
obtained_point_B = 0
obtained_point_C = 0

In [264]:
# 各クライアントの考える価値を数値化し、表に加える。
# 指定した視聴率/そのクライアントの号数価格
sample['value_A'] = round((sample['rating_A']/sample['price_A'])*100, 2)
sample['value_B'] = round((sample['rating_B']/sample['price_B'])*100, 2)
sample['value_C'] = round((sample['rating_C']/sample['price_C'])*100, 2)

In [296]:
print('client_Aの交換前の合計価格:', o_price_a, 'client_Aの交換前の合計視聴率', o_point_a)
print('client_Bの交換前の合計価格:', o_price_b, 'client_Bの交換前の合計視聴率', o_point_b)
print('client_Cの交換前の合計価格:', o_price_c, 'client_Cの交換前の合計視聴率', o_point_c)
sample.head(3)

client_Aの交換前の合計価格: 6034 client_Aの交換前の合計視聴率 59.20000000000001
client_Bの交換前の合計価格: 5821 client_Bの交換前の合計視聴率 60.8
client_Cの交換前の合計価格: 7137 client_Cの交換前の合計視聴率 74.19999999999999


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,10,357,372,362,3.7,3.3,3.7,1.04,0.89,1.02
1,13,607,627,608,6.1,7.0,5.1,1.00,1.12,0.84
2,15,230,212,218,2.4,2.0,2.5,1.04,0.94,1.15


***
***

### ここからアルゴリズム。

In [330]:
# ナップザック問題。

# size = 値段のデータ。
# weight = 価値のデータ。
# capacity = 元々の値段×1.05倍の値段。
# ids = 取得すべきだと考えられた枠のid    

def Knapsack(size, weight, capacity, obtained=0, rate=0.05):
    if capacity*(1 + rate) -  obtained > 0:        
        x = cvxpy.Variable(size.shape[0], boolean=True)
        # 要素の個数(これを加えて平均を最大化しないと、価格の小さいものばかりをとってしまう。)
        count = sum(np.ones(len(size)) * x)
        # 目的
        objective = cvxpy.Maximize(weight * x / count)
        # 制限。ここでは、capacity(価格)の上下5%を取っている。
        constraints = [capacity*(1 + rate) -  obtained >= size*x]
        constraints += [capacity*(1 - rate) - obtained <= size * x]

        prob = cvxpy.Problem(objective, constraints)
        prob.solve(solver=cvxpy.ECOS_BB)
        result = [round(ix, 0) for ix in x.value]

        return result
    else:
        return []

In [267]:
size_A = np.array(sample['price_A'])
weight_A = np.array(sample['value_A'])
capacity_A = o_price_a

ids_a = Knapsack(size_A, weight_A, capacity_A,  obtained_price_A)

In [268]:
size_B = np.array(sample['price_B'])
weight_B = np.array(sample['value_B'])
capacity_B = o_price_b

ids_b = Knapsack(size_B, weight_B, capacity_B,  obtained_price_B)

In [269]:
size_C = np.array(sample['price_C'])
weight_C = np.array(sample['value_C'])
capacity_C = o_price_c

# 今回のCのように、capacityの値を制約しすぎると、その値の枠に収まらなくなる可能性がある。
# その場合は、±の幅を大きくすれば良い。
ids_c = Knapsack(size_C, weight_C, capacity_C,  obtained_price_C, rate=0.1)

In [270]:
# 視聴率データのインデックス。
ids = np.array(sample['id'].index)

In [271]:
# それぞれが選んだ枠のid『のindex』が選択されている。
print(sum(ids_a),'個の枠:', ids_a * ids)
print(sum(ids_b),'個の枠:', ids_b * ids)
print(sum(ids_c),'個の枠:', ids_c * ids)

6.0 個の枠: [-0. -0.  2. -0. -0.  5. -0. -0. -0. -0. -0. -0. -0. 13. -0. -0. 16. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. 27. 28. -0.]
5.0 個の枠: [ 0.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 13.  0.  0. 16.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0. 27. 28.  0.]
7.0 個の枠: [ 0.  1.  2.  0.  0.  0. -0. -0. -0.  0. -0. -0. -0. 13.  0.  0. 16. -0.
 -0. -0. 20.  0.  0. -0. -0. -0. -0. 27. 28. -0.]


In [272]:
# それぞれの枠を選択したクライアントの中から１つをランダムに選択する。
ids_dict = dict()
for i in ids:
    values = []
    if i in ids_a * ids:
        values.append('a')
    if i in ids_b * ids:
        values.append('b')
    if i in ids_c * ids:
        values.append('c')
    if values:
        value = random.choice(values)
        ids_dict[i] = value

In [273]:
# クライアントごとに、取得することのできた枠のid『のindex』をリスト化する。
a_add_ids = [ids for ids, client in ids_dict.items() if client == 'a']
b_add_ids = [ids for ids, client in ids_dict.items() if client == 'b']
c_add_ids = [ids for ids, client in ids_dict.items() if client == 'c']

# それらの和集合（＝抜き出すもの）
extraction_ids = a_add_ids + b_add_ids + c_add_ids

In [274]:
# 取得することのできた枠のid
print("client_a:", a_add_ids)
print("client_b:", b_add_ids)
print("client_c:", c_add_ids)

client_a: [2, 5, 16, 27, 28]
client_b: []
client_c: [0, 1, 13, 20]


In [275]:
# 取得した枠の値段を足す。
obtained_price_A += sum(sample.query('index in ' + str(a_add_ids))['price_A'])
obtained_price_B += sum(sample.query('index in ' + str(b_add_ids))['price_B'])
obtained_price_C += sum(sample.query('index in ' + str(c_add_ids))['price_C'])

In [276]:
# 取得した枠の視聴率を足す。
obtained_point_A += sum(sample.query('index in ' + str(a_add_ids))['rating_A'])
obtained_point_B += sum(sample.query('index in ' + str(b_add_ids))['rating_B'])
obtained_point_C += sum(sample.query('index in ' + str(c_add_ids))['rating_C'])

In [277]:
# 取得された枠を削除する。
sample = sample.query('index not in ' + str(extraction_ids)).reset_index(drop=True)

In [278]:
print('残りの枠:', len(sample))
print('client_Aの合計価格:', obtained_price_A, 'client_Aの合計視聴率', obtained_point_A)
print('client_Bの合計価格:', obtained_price_B, 'client_Bの合計視聴率', obtained_point_B)
print('client_Cの合計価格:', obtained_price_C, 'client_Cの合計視聴率', obtained_point_C)
sample.head(3)

残りの枠: 21


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,4,217,237,227,2.3,1.9,2.3,1.06,0.80,1.01
1,5,167,149,178,1.5,1.5,2.0,0.90,1.01,1.12
2,7,757,743,769,7.9,7.6,7.2,1.04,1.02,0.94


***
***

### アルゴリズム２回目

In [280]:
size_A = np.array(sample['price_A'])
weight_A = np.array(sample['value_A'])
capacity_A = o_price_a

ids_a = Knapsack(size_A, weight_A, capacity_A,  obtained_price_A)

In [281]:
size_B = np.array(sample['price_B'])
weight_B = np.array(sample['value_B'])
capacity_B = o_price_b

ids_b = Knapsack(size_B, weight_B, capacity_B,  obtained_price_B)

In [283]:
size_C = np.array(sample['price_C'])
weight_C = np.array(sample['value_C'])
capacity_C = o_price_c

ids_c = Knapsack(size_C, weight_C, capacity_C,  obtained_price_C)

In [284]:
# 視聴率データのインデックス。
ids = np.array(sample['id'].index)

In [285]:
# それぞれが選んだ枠のid『のindex』が選択されている。
print(sum(ids_a),'個の枠:', ids_a * ids)
print(sum(ids_b),'個の枠:', ids_b * ids)
print(sum(ids_c),'個の枠:', ids_c * ids)

2.0 個の枠: [-0.  1. -0. -0. -0. -0. -0.  7. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0.]
8.0 個の枠: [-0. -0.  2.  3.  4.  0.  6.  7.  0. -0. -0.  0. 12.  0. -0.  0. 16.  0.
 18.  0.  0.]
5.0 個の枠: [ 0.  0.  2.  3.  4.  0.  6.  7.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 -0.  0.  0.]


In [286]:
# それぞれの枠を選択したクライアントの中から１つをランダムに選択する。
ids_dict = dict()
for i in ids:
    values = []
    if i in ids_a * ids:
        values.append('a')
    if i in ids_b * ids:
        values.append('b')
    if i in ids_c * ids:
        values.append('c')
    if values:
        value = random.choice(values)
        ids_dict[i] = value

In [287]:
# クライアントごとに、取得することのできた枠のid『のindex』をリスト化する。
a_add_ids = [ids for ids, client in ids_dict.items() if client == 'a']
b_add_ids = [ids for ids, client in ids_dict.items() if client == 'b']
c_add_ids = [ids for ids, client in ids_dict.items() if client == 'c']

# それらの和集合（＝抜き出すもの）
extraction_ids = a_add_ids + b_add_ids + c_add_ids

In [288]:
# 取得することのできた枠のid
print("client_a:", a_add_ids)
print("client_b:", b_add_ids)
print("client_c:", c_add_ids)

client_a: [1]
client_b: [0, 2, 4, 6, 12, 16, 18]
client_c: [3, 7]


In [289]:
# 取得した枠の値段を足す。
obtained_price_A += sum(sample.query('index in ' + str(a_add_ids))['price_A'])
obtained_price_B += sum(sample.query('index in ' + str(b_add_ids))['price_B'])
obtained_price_C += sum(sample.query('index in ' + str(c_add_ids))['price_C'])

In [290]:
# 取得した枠の視聴率を足す。
obtained_point_A += sum(sample.query('index in ' + str(a_add_ids))['rating_A'])
obtained_point_B += sum(sample.query('index in ' + str(b_add_ids))['rating_B'])
obtained_point_C += sum(sample.query('index in ' + str(c_add_ids))['rating_C'])

In [291]:
# 取得された枠を削除する。
sample = sample.query('index not in ' + str(extraction_ids)).reset_index(drop=True)

In [295]:
print('残りの枠:', len(sample))
print('client_Aの合計価格:', obtained_price_A, 'client_Aの合計視聴率', obtained_point_A)
print('client_Bの合計価格:', obtained_price_B, 'client_Bの合計視聴率', obtained_point_B)
print('client_Cの合計価格:', obtained_price_C, 'client_Cの合計視聴率', obtained_point_C)
sample.head(3)

残りの枠: 11
client_Aの合計価格: 4914 client_Aの合計視聴率 49.800000000000004
client_Bの合計価格: 4407 client_Bの合計視聴率 44.4
client_Cの合計価格: 4926 client_Cの合計視聴率 49.6


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,10,357,372,362,3.7,3.3,3.7,1.04,0.89,1.02
1,13,607,627,608,6.1,7.0,5.1,1.00,1.12,0.84
2,15,230,212,218,2.4,2.0,2.5,1.04,0.94,1.15


***
***

### アルゴリズム３回目

In [297]:
size_A = np.array(sample['price_A'])
weight_A = np.array(sample['value_A'])
capacity_A = o_price_a

ids_a = Knapsack(size_A, weight_A, capacity_A,  obtained_price_A)

In [298]:
size_B = np.array(sample['price_B'])
weight_B = np.array(sample['value_B'])
capacity_B = o_price_b

ids_b = Knapsack(size_B, weight_B, capacity_B,  obtained_price_B)

In [299]:
size_C = np.array(sample['price_C'])
weight_C = np.array(sample['value_C'])
capacity_C = o_price_c

ids_c = Knapsack(size_C, weight_C, capacity_C,  obtained_price_C)

In [300]:
# 視聴率データのインデックス。
ids = np.array(sample['id'].index)

In [301]:
# それぞれが選んだ枠のid『のindex』が選択されている。
print(sum(ids_a),'個の枠:', ids_a * ids)
print(sum(ids_b),'個の枠:', ids_b * ids)
print(sum(ids_c),'個の枠:', ids_c * ids)

2.0 個の枠: [ 0. -0.  0.  0.  0.  0.  0.  0.  8.  0. 10.]
2.0 個の枠: [ 0.  1. -0.  0.  0.  0.  0.  0.  0.  0. 10.]
4.0 個の枠: [ 0.  1.  0.  0.  0.  0.  0.  0.  8.  9. 10.]


In [302]:
# それぞれの枠を選択したクライアントの中から１つをランダムに選択する。
ids_dict = dict()
for i in ids:
    values = []
    if i in ids_a * ids:
        values.append('a')
    if i in ids_b * ids:
        values.append('b')
    if i in ids_c * ids:
        values.append('c')
    if values:
        value = random.choice(values)
        ids_dict[i] = value

In [303]:
# クライアントごとに、取得することのできた枠のid『のindex』をリスト化する。
a_add_ids = [ids for ids, client in ids_dict.items() if client == 'a']
b_add_ids = [ids for ids, client in ids_dict.items() if client == 'b']
c_add_ids = [ids for ids, client in ids_dict.items() if client == 'c']

# それらの和集合（＝抜き出すもの）
extraction_ids = a_add_ids + b_add_ids + c_add_ids

In [304]:
# 取得することのできた枠のid
print("client_a:", a_add_ids)
print("client_b:", b_add_ids)
print("client_c:", c_add_ids)

client_a: [8]
client_b: []
client_c: [0, 1, 9, 10]


In [305]:
# 取得した枠の値段を足す。
obtained_price_A += sum(sample.query('index in ' + str(a_add_ids))['price_A'])
obtained_price_B += sum(sample.query('index in ' + str(b_add_ids))['price_B'])
obtained_price_C += sum(sample.query('index in ' + str(c_add_ids))['price_C'])

In [306]:
# 取得した枠の視聴率を足す。
obtained_point_A += sum(sample.query('index in ' + str(a_add_ids))['rating_A'])
obtained_point_B += sum(sample.query('index in ' + str(b_add_ids))['rating_B'])
obtained_point_C += sum(sample.query('index in ' + str(c_add_ids))['rating_C'])

In [307]:
# 取得された枠を削除する。
sample = sample.query('index not in ' + str(extraction_ids)).reset_index(drop=True)

In [312]:
print('残りの枠:', len(sample))
print('client_Aの合計価格:', obtained_price_A, 'client_Aの合計視聴率', obtained_point_A)
print('client_Bの合計価格:', obtained_price_B, 'client_Bの合計視聴率', obtained_point_B)
print('client_Cの合計価格:', obtained_price_C, 'client_Cの合計視聴率', obtained_point_C)
display(sample.head(3))

残りの枠: 6
client_Aの合計価格: 5521 client_Aの合計視聴率 55.400000000000006
client_Bの合計価格: 4407 client_Bの合計視聴率 44.4
client_Cの合計価格: 7068 client_Cの合計視聴率 68.80000000000001


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,15,230,212,218,2.4,2.0,2.5,1.04,0.94,1.15
1,16,333,323,337,3.0,4.2,2.8,0.90,1.30,0.83
2,18,413,401,429,4.0,4.0,4.4,0.97,1.00,1.03


***
***

### アルゴリズム４回目

In [313]:
size_A = np.array(sample['price_A'])
weight_A = np.array(sample['value_A'])
capacity_A = o_price_a

ids_a = Knapsack(size_A, weight_A, capacity_A,  obtained_price_A)

In [314]:
size_B = np.array(sample['price_B'])
weight_B = np.array(sample['value_B'])
capacity_B = o_price_b

ids_b = Knapsack(size_B, weight_B, capacity_B,  obtained_price_B)

In [315]:
size_C = np.array(sample['price_C'])
weight_C = np.array(sample['value_C'])
capacity_C = o_price_c

ids_c = Knapsack(size_C, weight_C, capacity_C,  obtained_price_C)

In [316]:
# 視聴率データのインデックス。
ids = np.array(sample['id'].index)

In [317]:
# それぞれが選んだ枠のid『のindex』が選択されている。
print(sum(ids_a),'個の枠:', ids_a * ids)
print(sum(ids_b),'個の枠:', ids_b * ids)
print(sum(ids_c),'個の枠:', ids_c * ids)

1.0 個の枠: [ 0.  0. -0. -0.  0.  0.]
4.0 個の枠: [0. 1. 2. 3. 0. 5.]
0.0 個の枠: [-0. -0. -0. -0. -0. -0.]


In [318]:
# それぞれの枠を選択したクライアントの中から１つをランダムに選択する。
ids_dict = dict()
for i in ids:
    values = []
    if i in ids_a * ids:
        values.append('a')
    if i in ids_b * ids:
        values.append('b')
    if i in ids_c * ids:
        values.append('c')
    if values:
        value = random.choice(values)
        ids_dict[i] = value

In [319]:
# クライアントごとに、取得することのできた枠のid『のindex』をリスト化する。
a_add_ids = [ids for ids, client in ids_dict.items() if client == 'a']
b_add_ids = [ids for ids, client in ids_dict.items() if client == 'b']
c_add_ids = [ids for ids, client in ids_dict.items() if client == 'c']

# それらの和集合（＝抜き出すもの）
extraction_ids = a_add_ids + b_add_ids + c_add_ids

In [320]:
# 取得することのできた枠のid
print("client_a:", a_add_ids)
print("client_b:", b_add_ids)
print("client_c:", c_add_ids)

client_a: [0]
client_b: [1, 2, 3, 5]
client_c: []


In [321]:
# 取得した枠の値段を足す。
obtained_price_A += sum(sample.query('index in ' + str(a_add_ids))['price_A'])
obtained_price_B += sum(sample.query('index in ' + str(b_add_ids))['price_B'])
obtained_price_C += sum(sample.query('index in ' + str(c_add_ids))['price_C'])

In [322]:
# 取得した枠の視聴率を足す。
obtained_point_A += sum(sample.query('index in ' + str(a_add_ids))['rating_A'])
obtained_point_B += sum(sample.query('index in ' + str(b_add_ids))['rating_B'])
obtained_point_C += sum(sample.query('index in ' + str(c_add_ids))['rating_C'])

In [323]:
# 取得された枠を削除する。
sample = sample.query('index not in ' + str(extraction_ids)).reset_index(drop=True)

In [324]:
print('残りの枠:', len(sample))
print('client_Aの合計価格:', obtained_price_A, 'client_Aの合計視聴率', obtained_point_A)
print('client_Bの合計価格:', obtained_price_B, 'client_Bの合計視聴率', obtained_point_B)
print('client_Cの合計価格:', obtained_price_C, 'client_Cの合計視聴率', obtained_point_C)
display(sample.head(3))

残りの枠: 1
client_Aの合計価格: 5751 client_Aの合計視聴率 57.800000000000004
client_Bの合計価格: 5906 client_Bの合計視聴率 61.9
client_Cの合計価格: 7068 client_Cの合計視聴率 68.80000000000001


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,22,287,273,269,2.6,2.5,3.5,0.91,0.92,1.3


In [328]:
print('client_Aの交換前の合計価格:', o_price_a, 'client_Aの交換前の合計視聴率', o_point_a)
print('client_Bの交換前の合計価格:', o_price_b, 'client_Bの交換前の合計視聴率', o_point_b)
print('client_Cの交換前の合計価格:', o_price_c, 'client_Cの交換前の合計視聴率', o_point_c)

client_Aの交換前の合計価格: 6034 client_Aの交換前の合計視聴率 59.20000000000001
client_Bの交換前の合計価格: 5821 client_Bの交換前の合計視聴率 60.8
client_Cの交換前の合計価格: 7137 client_Cの交換前の合計視聴率 74.19999999999999


***
***

In [359]:
# データフレームの長さが１だとうまく動作しないので、全て０のカラムを足す。
add_df = pd.DataFrame(np.zeros(10)).T
add_df.columns = sample.columns
add_df

,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [360]:
sample = pd.concat([sample, add_df]).reset_index(drop=True)

In [361]:
sample.head(3)

,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,22.0,287.0,273.0,269.0,2.6,2.5,3.5,0.91,0.92,1.3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0


***
***

### アルゴリズム５回目

In [362]:
size_A = np.array(sample['price_A'])
weight_A = np.array(sample['value_A'])
capacity_A = o_price_a

ids_a = Knapsack(size_A, weight_A, capacity_A,  obtained_price_A, )

In [363]:
size_B = np.array(sample['price_B'])
weight_B = np.array(sample['value_B'])
capacity_B = o_price_b

ids_b = Knapsack(size_B, weight_B, capacity_B,  obtained_price_B)

In [364]:
size_C = np.array(sample['price_C'])
weight_C = np.array(sample['value_C'])
capacity_C = o_price_c

ids_c = Knapsack(size_C, weight_C, capacity_C,  obtained_price_C)

In [365]:
# 視聴率データのインデックス。
ids = np.array(sample['id'].index)

In [366]:
# それぞれが選んだ枠のid『のindex』が選択されている。
print(sum(ids_a),'個の枠:', ids_a * ids)
print(sum(ids_b),'個の枠:', ids_b * ids)
print(sum(ids_c),'個の枠:', ids_c * ids)

1.0 個の枠: [0. 0.]
0.0 個の枠: [-0.  0.]
0.0 個の枠: [0. 0.]


In [367]:
# それぞれの枠を選択したクライアントの中から１つをランダムに選択する。
ids_dict = dict()
for i in ids:
    values = []
    if i in ids_a * ids:
        values.append('a')
    if i in ids_b * ids:
        values.append('b')
    if i in ids_c * ids:
        values.append('c')
    if values:
        value = random.choice(values)
        ids_dict[i] = value

In [368]:
# クライアントごとに、取得することのできた枠のid『のindex』をリスト化する。
a_add_ids = [ids for ids, client in ids_dict.items() if client == 'a']
b_add_ids = [ids for ids, client in ids_dict.items() if client == 'b']
c_add_ids = [ids for ids, client in ids_dict.items() if client == 'c']

# それらの和集合（＝抜き出すもの）
extraction_ids = a_add_ids + b_add_ids + c_add_ids

In [369]:
# 取得することのできた枠のid
print("client_a:", a_add_ids)
print("client_b:", b_add_ids)
print("client_c:", c_add_ids)

client_a: [0]
client_b: []
client_c: []


In [370]:
# 取得した枠の値段を足す。
obtained_price_A += sum(sample.query('index in ' + str(a_add_ids))['price_A'])
obtained_price_B += sum(sample.query('index in ' + str(b_add_ids))['price_B'])
obtained_price_C += sum(sample.query('index in ' + str(c_add_ids))['price_C'])

In [371]:
# 取得した枠の視聴率を足す。
obtained_point_A += sum(sample.query('index in ' + str(a_add_ids))['rating_A'])
obtained_point_B += sum(sample.query('index in ' + str(b_add_ids))['rating_B'])
obtained_point_C += sum(sample.query('index in ' + str(c_add_ids))['rating_C'])

In [372]:
# 取得された枠を削除する。
sample = sample.query('index not in ' + str(extraction_ids)).reset_index(drop=True)

In [373]:
print('残りの枠:', len(sample))
print('client_Aの合計価格:', obtained_price_A, 'client_Aの合計視聴率', obtained_point_A)
print('client_Bの合計価格:', obtained_price_B, 'client_Bの合計視聴率', obtained_point_B)
print('client_Cの合計価格:', obtained_price_C, 'client_Cの合計視聴率', obtained_point_C)
display(sample.head(3))

残りの枠: 1
client_Aの合計価格: 6038.0 client_Aの合計視聴率 60.400000000000006
client_Bの合計価格: 5906 client_Bの合計視聴率 61.9
client_Cの合計価格: 7068 client_Cの合計視聴率 68.80000000000001


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


***
***

In [374]:
# データフレーム先ほど足したものだけになったので、ここで終了にする。
sample == add_df

,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,True,True,True,True,True,True,True,True,True,True


### ○問題点
1. 合計の視聴率を最大にすることを目標にしていないこと。
2. indexを利用した結果、0番目を選択していなくても値が同じになるため、選択したものと考えられてしまう。
3. 視聴率和が元々の視聴率和を超えた段階でSTOPして良いと考えられる。